# Conditional Random Fields (CRF)
## Introduction

In this notebook we're gonna attempt to train a CRF model and use it to identify Named Entities. In the last notebook (HMM), we trained a Hidden Markov Model, which is a generative model based on directed graphs, to identify Named Entities. Now we're gonna try out a more generalized model which is based on undirected graph. 

In this specific case, we are going to see the Linear-chain CRF which is a special case of CRF very similar to HMM. In fact, we may consider HMM as a special case of a linear-chain CRF. The graph in a linear-chain CRF, just as HMM is a sequence of nodes that produce symbols, but the difference is that CRF take into account the whole set of observations to produce the symbols. 

The library that we are going to use is the sklearn-crfsuite (http://sklearn-crfsuite.readthedocs.io/), which is a sklearn wrapper over the origin crfsuite library http://www.chokkan.org/software/crfsuite/ implemented in C++. 

## Brief Review of CRF

Before getting into the code, let's just review the CRF definition and how to train it.

### CRF definition

(Summary based on this paper: https://people.cs.umass.edu/~mccallum/papers/crf-tutorial.pdf )

The intuition behind the CRF is that we can estimate p(x|y) directly instead of going through p(x,y). CRF does this by using feature-functions that consider the transition between the two states $y_t, y_{t-1}$ and a vector $\boldsymbol{x_t}$ containing all the global components of $\boldsymbol{x}$ (aka features) that are needed to compute the feature function at time t. These feature-functions return a real number which is then multiplied by a weight parameter $\lambda_k$.


The formal definition is:

$$ p(x|y) = \frac{1}{Z(x)}\exp[\sum_{k=1}^{K}\lambda_k f_k(y_t, y_{t-1}, \boldsymbol{x_t} )  ]   $$
where Z(x) is a normalizing factor defined:  
$$ Z(x) = exp[\sum_y \lambda_k f_k(y_t, y_{t-1}, x_t )   ]  $$

Note that the feature functions can be virtually anything, but commonly they are defined as product of binary functions that indicate the presence or absence of an attribute. In which case the functions might take this form:

$$ F_j(x,y) =  A_a(x) B_b(y) $$ 
where the subscript $a$ indexes a set of functions of $x$ and $b$ indexes a set of functions of $y$.

For example there could be a set of indicator functions of x:
- $A_1(x) = I(x$ starts with a Capital Letter$)$
- $A_2(x) = I(len(x) > 6)$
- $A_3(x) = I(x$[0:4] == 'Wash'$)$

(source: http://cseweb.ucsd.edu/~elkan/250Bwinter2012/loglinearCRFs.pdf )

### Parameter Estimation (training)
So, for constructing the model we have the labels or y values (which would be the tags), the x values or the features,
and the feature functions which are defined arbitrary (it could be a product of indicator functions). What's missing are the $\lambda$ parameters (or weights). These $\lambda$ values are important since they determine how relevant a feature function is. 

For completing the model we are going to estimate the parameters (aka train the model) using the traditional Maximum Likelihood.

We can frame the task as: given a set of training samples, find the parameter values $\lambda_k$ that maximize the conditional probability of the samples. In other words, the goal is to find the $\lambda_k$  maximize the log likelihood or conditional log likelihood with an added regularization term (i.e. to smooth).

$$ \boldsymbol\lambda^* = \arg_{\lambda} min L(\boldsymbol\lambda|x) + C\frac{1}{2}||\lambda^2|| $$

where 

$$L(\boldsymbol\lambda| x) =- log \prod_{i=1}^{N}  p(y^{(i)}| x^{(i)}; \lambda ) = -\sum_{i=1}^{N}log  p(y^{(i)}| x^{(i)};\lambda )$$

**This function is convex i.e. with no local minimums**  so we can use gradient based methods like Stochastic Gradient Ascent. The difficulty is that in order to do it, it is necessary to calculate all the partial derrivatives with respect to each feature function, thus making it computationally expensive. One solution is to use the forward-backward algorithm to estimate the partial derivatives.

There are also other alternatives to estimate the parameters using Newtowns Method to converge faster or quasi-Newtown methods such as BFGS or L-BFGS. There's also an interesting alternative using a perceptron.

### Inference (Predicting)

Finally, having the model complete the last step is to use it to predict labels. This is, finding the labels that maximize the conditional probability:
$$ \boldsymbol y^∗ = \underset{y}{\operatorname{argmax}} p(y|x) $$

Since we are working with linear-chain CRF then a variation of dynamic-programming algorithms used for HMM can be employed. A popular algorithm is the recursive Viterbri algorithm. The recursive part is defined as:

$$ \delta_t(j) = \underset{i \in S}{\operatorname{max}} \Psi_t(j,i, x_t)\delta_t(i) $$

where $t$ is the time-step, j is the current state, S is the set of all states, $\Psi_t$ is a way to write the set of feature functions at time step $t$.

# Tutorial

Now that we've reviewed the CRF definition, we're going to code. Specifically we are going to follow the tutorial from the sklearn-crfsuite but with small twists and changes to check if we really are learning how to use it.

The tutorial is from this site: http://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html

In [1]:
#import the libraries for the project
import pandas as pd
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import conll2002

###  Dataset
First we are going to import the datasets. For this tutorial we are using the spanish set of Conll2002, which conviniently has the train and test datasets splitted.

In [2]:
train_sents = list(conll2002.iob_sents('esp.train'))
test_sents = list(conll2002.iob_sents('esp.testb'))

print "Number of train sentences: %i" %len(train_sents)
print "Number of test sentences: %i" %len(test_sents)

Number of train sentences: 8323
Number of test sentences: 1517


Let's just have a sneak peek to the data to get an idea of how the data looks.

In [3]:
train_sents[0]

[(u'Melbourne', u'NP', u'B-LOC'),
 (u'(', u'Fpa', u'O'),
 (u'Australia', u'NP', u'B-LOC'),
 (u')', u'Fpt', u'O'),
 (u',', u'Fc', u'O'),
 (u'25', u'Z', u'O'),
 (u'may', u'NC', u'O'),
 (u'(', u'Fpa', u'O'),
 (u'EFE', u'NC', u'B-ORG'),
 (u')', u'Fpt', u'O'),
 (u'.', u'Fp', u'O')]

As we can see each sentence is composed by a list of tuples. Each tuple has three fields: 
1. **Word**: This is the actual word in the sentence.
2. **Part-of-Speech tag (POS)**: It indicates the "role" of the word in the sentence
3. **IOB Named entity tag** : A tag that shows which words are part of a named entity and their entity type using the Inside Outside Beginning (IOB) notation. The IOB notation basically has two parts: 
    - The first letter indicates if the word is the beggining of an entity (B), the inner part of an entity (I) or if it's a word outside the entitities (O).
    - Then the word following the dash (-) indicates the entity type. For example, LOC means that the entity is a location.
    
    
For this tutorial we are trying to figure out the IOB tags in order to extract the named entities.

### Feature engineering

The next step is to create a set of features from the original dataset to feed the training algorithm with rich information. 

The tutorial gives a nice implementation for generating the features with the sent2features and word2features functions. Since I don't want to just copy the code I'm gonna rewrite those functions in another way to get the concept right.

The basic idea is to generate an List of Lists with dictionary objects that contain all the feature values. Conviently using dictionaries allow us to feed the trainer with only the relevant features at each step. Recall the CRF definition
> CRF does this by using feature-functions that consider the transition between the two states $y_t, y_{t-1}$ and a vector $\boldsymbol{x_t}$ containing all the global components of $\boldsymbol{x}$ (aka features) that are needed to compute the feature function at time t.

Notice in the code below that for the first word in a sentence we don't use some features based on the preceding word (for obvious reasons) and for the rest of the words they don't have the "first_word" feature. This is our implementation for this example, but actually we can define features using any component or from the whole sequence if we want.

In [4]:
def getXY(sentences):
    feature_sents = []
    labels = []
    for sent in sentences:
        f_sent = []
        for idx, item in enumerate(sent):
            word = item[0]
            pos_tag = item[1]
            
            features = {
                'word': word.lower(),  #the word itself
                'suffix_3': word[-3:], #the word suffix 
                'suffix_2': word[-2:], #the word suffix 
                'upper': word.isupper(),
                'title': word.istitle(), #True if the first word is upper but the rest is lower
                'digit': word.isdigit(),
                'postag': pos_tag,
                'postag_base': pos_tag[:2],
            }
            
            if idx == 0:                
                features['first_word'] = True
            else:
                word_1 = sent[idx-1][0]
                postag_1 = sent[idx-1][1]
                
                features.update({
                    "word_1": word_1.lower(),
                    'word_1_title()': word_1.istitle(),
                    'word_1_upper': word_1.isupper(),
                    'word_1_postag': postag_1,
                    'word_1_postag_base': postag_1[:2],  
                })
            
            if idx == len(sent) - 1:
                features["last_word"] = True
            #append the features to the sentence
            f_sent.append(features)
            
        #append the sentences to the list
        feature_sents.append(f_sent)
        #apend labels 
        labels.append( [ label for w,p,label in sent  ] )
        
    return feature_sents, labels    

Executing the function to get the features and label lists.

In [5]:
X_train, y_train = getXY(train_sents)
X_test, y_test = getXY(test_sents)

Let's just visualize one item of a sentence to see check the result:    
**(Notice that the first and second have different features)**

In [10]:
#first word of sentence 0
X_train[0][1]

{'digit': False,
 'postag': u'Fpa',
 'postag_base': u'Fp',
 'suffix_2': u'(',
 'suffix_3': u'(',
 'title': False,
 'upper': False,
 'word': u'(',
 'word_1': u'melbourne',
 'word_1_postag': u'NP',
 'word_1_postag_base': u'NP',
 'word_1_title()': True,
 'word_1_upper': False}

In [11]:
# second word of sentence 0
X_train[0][1]

{'digit': False,
 'postag': u'Fpa',
 'postag_base': u'Fp',
 'suffix_2': u'(',
 'suffix_3': u'(',
 'title': False,
 'upper': False,
 'word': u'(',
 'word_1': u'melbourne',
 'word_1_postag': u'NP',
 'word_1_postag_base': u'NP',
 'word_1_title()': True,
 'word_1_upper': False}

Let's check the corresponding label:

In [7]:
y_test[0][1]

u'I-LOC'

# Training

Ok, now the fun part: Training the model. Since this specific library provides an API compatible to sklearn this part is really easy.

We just instantiate the classifier and then we fit (train) the data as any classifier in sklearn.

The parameters for the Classifier are:
- **algorithm**: The parameter estimation algorithm to use. For this tutorial we are using the lbfgs. 
- **c1**:The coefficient for L1 regularization applied to the lbfgs.
- **c2**:The coefficient for L2 regularization applied to the lbfgs
- **max_iterations**: Limits the number of iterations for the optimization algorithms.
- **all_possible_transitions**: generates transition features that don't occur in the dataset.

There are other parameters which are described here: 
http://sklearn-crfsuite.readthedocs.io/en/latest/api.html

We are also going to check the other parameters later in this notebook. Right now we just need to learn how to use it.

In [39]:
%%time
clf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
clf.fit(X_train, y_train)

CPU times: user 39.7 s, sys: 416 ms, total: 40.1 s
Wall time: 41.4 s


## Testing

Let's check how the classifier performs with the test data.  Since we are just interested on the named entities we are going to only include the B and I labels in the score.

Notice that for scoring we are using the flat_classification_report() method that comes in the sklearn_crfsuite.metrics module. This method prints exactly the same output as the sklearn classification_report(), but as input it receives a List of Lists.

In [40]:
#predict
y_pred = clf.predict(X_test)

#prepare the labels by removing the O labels
labels = list(clf.classes_)
labels.remove('O')
print metrics.flat_classification_report(y_test, y_pred, labels=sorted(labels), digits=3 )

             precision    recall  f1-score   support

      B-LOC      0.803     0.759     0.780      1084
     B-MISC      0.700     0.537     0.608       339
      B-ORG      0.813     0.838     0.825      1400
      B-PER      0.817     0.865     0.841       735
      I-LOC      0.695     0.637     0.665       325
     I-MISC      0.661     0.542     0.596       557
      I-ORG      0.840     0.799     0.819      1104
      I-PER      0.880     0.938     0.908       634

avg / total      0.797     0.777     0.786      6178



As we can observe the model performs well with the initial parameters and the simple features.

## Tuning

Ok, so now that we know how to train and test the model, let's improve the model by tuning it's hyper parameters. 

For this, we will use a CVGridSearch provided by the sklearn library. (Good thing that this CRF is compatible with sklearn). 

First we are going to create a function that we can re-use to test various parameters. The function will create a scorer (f1 scorer with only the labels of our interest), instantiate and train the model using CVSearchGrid and  print the results. 

In [26]:
#re-usable function to train and tune the classifier.
def train_tune_classifier(parameters, X, y, verbose=1):
    #scorer
    f1_scorer = make_scorer( metrics.flat_f1_score,
                             average='weighted', labels=labels)
    #model instance
    crf = sklearn_crfsuite.CRF()

    clf = GridSearchCV(crf, parameters, scoring=f1_scorer, cv=4, n_jobs=4, verbose = verbose )
    clf.fit(X, y)
    
    #print results
    print "Best Parameters: %s" %clf.best_params_
    print "Best Score: %0.4f" %clf.best_score_

    #dataframe with detailed results 
    params = clf.cv_results_["params"]
    df = pd.DataFrame({
            "params": [ ", ".join([str(param[x]) for x in param]) for param in params ],
            "mean_train_score": [x for x in clf.cv_results_["mean_train_score"]],
            "mean_test_score": [x for x in clf.cv_results_["mean_test_score"]],
            "mean_score_time": [x for x in clf.cv_results_["mean_score_time"]],
            "mean_fit_time": [x for x in clf.cv_results_["mean_fit_time"]]
        }).sort_values(by="mean_test_score",ascending=False)
    
    return clf, df


### Parameter estimation algorithms

Now that we have the function, we are going to test the different parameter estimation algorithms with their default hyper-parameters. Let's recall that the idea is to maximize the log likelihood and that there are various approaches to compute it. CRFSuite supports the following:
- lbfgs: Gradient Descent using the Limited BFGS method.
- l2sgd: Stochastic Descent using with a L2 regularization term (i.e. to smooth).
- ap: Average Perceptron. 
- pa: Passive Agressive.
- arow: Adaptive Regularization Of Weight Vector

Let's run the code and see.. (if you are actually running this, grab a coffee , read the news cause it might take a while... oh and also expect this to consume some big portion of RAM)

In [27]:
#parameters
parameters = {
    "algorithm": ['lbfgs','l2sgd','ap','pa','arow'],
    "max_iterations": [100]
}

#tune
clf, results = train_tune_classifier(parameters, X_train, y_train, verbose=1)
results

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  9.4min finished


Best Parameters: {'algorithm': 'pa', 'max_iterations': 100}
Best Score: 0.7579


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,params
3,20.869644,0.646210,0.757869,0.967718,"pa, 100"
2,19.214253,0.645312,0.756501,0.970804,"ap, 100"
0,32.225473,0.963040,0.743637,0.880462,"lbfgs, 100"
1,33.748841,1.443953,0.740227,0.882993,"l2sgd, 100"
4,18.001748,0.621078,0.691417,0.967923,"arow, 100"


As we can observe the best scores are for the passive-agressive and the perceptron. Nevertheless they seem to be overfitting as the difference between train and test scores are larger than the ones for the gradient. 

Also, we can notice that the pa and ap algorithms are faster than the gradient based algorithms. So we are going to go with the passive-aggresive since it seems to be a good option in terms of score and time.

## Passive-Aggresive

The passive-aggresive algorithm is describe in this paper: http://www.jmlr.org/papers/volume7/crammer06a/crammer06a.pdf

The intuition behind the passive-aggresive algorithm is fairly simple. The algorithm makes many rounds were it looks at each item of the sequence and makes a prediction. After the round finishes, it receives feedback about the predictions and updates the weights only when it made a mistake (aggresive part) and it does nothing if it made a correct prediction (the passive part). The model is noise-resistant with has two main algorithm variants (PA-I and PA-II).

## Tuning with PA

Ok now that we've selected a maximization algorithm, let's tune its hyper parameters. We are going to tune:

- **pa_type**: The strategy for updating feature weights. There are three strategies
    - 0: PA  without slack variables
    - 1: PA-I
    - 2: PA-II
- **c**: Called the aggresiveness parameter it determines how "aggresive" the update step will be. (it only applies to PA-I and PA-II )
- **error_sensitive**: If True, the algorithm includes the squared root of the incorrect labels.
- **averaging**: If True, the algorithm computes the average of all weights at all update steps.

First we are going to choose a pa_type using the default parameters.

In [55]:
#parameters
parameters = {
    "algorithm": ['pa'],
    'pa_type': [0,1,2],
}

#tune
clf, results = train_tune_classifier(parameters, X_train, y_train, verbose=2)
results

Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] pa_type=0, algorithm=pa .........................................
[CV] .......................... pa_type=0, algorithm=pa, total=  21.0s
[CV] pa_type=0, algorithm=pa .........................................
[CV] .......................... pa_type=0, algorithm=pa, total=  20.3s
[CV] pa_type=0, algorithm=pa .........................................
[CV] .......................... pa_type=0, algorithm=pa, total=  20.7s
[CV] pa_type=0, algorithm=pa .........................................
[CV] .......................... pa_type=0, algorithm=pa, total=  20.8s
[CV] pa_type=1, algorithm=pa .........................................
[CV] .......................... pa_type=1, algorithm=pa, total=  21.7s
[CV] pa_type=1, algorithm=pa .........................................
[CV] .......................... pa_type=1, algorithm=pa, total=  21.3s
[CV] pa_type=1, algorithm=pa .........................................
[CV] ............

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  5.5min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  5.5min finished


Best Parameters: {'pa_type': 0, 'algorithm': 'pa'}
Best Score: 0.7590


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,params
0,20.077897,0.637137,0.758975,0.967500,"0, pa"
1,19.953407,0.968848,0.758620,0.967618,"1, pa"
2,19.928580,0.609879,0.758512,0.967650,"2, pa"


The best score is the PA without the slack variables. Let's evaluate it on the test dataset.

In [56]:
#predict
y_pred = clf.predict(X_test)
print metrics.flat_classification_report(y_test, y_pred, labels=sorted(labels), digits=3 )

             precision    recall  f1-score   support

      B-LOC      0.795     0.756     0.775      1084
     B-MISC      0.688     0.566     0.621       339
      B-ORG      0.815     0.836     0.825      1400
      B-PER      0.809     0.878     0.842       735
      I-LOC      0.740     0.640     0.686       325
     I-MISC      0.696     0.562     0.622       557
      I-ORG      0.854     0.794     0.823      1104
      I-PER      0.881     0.937     0.908       634

avg / total      0.803     0.780     0.790      6178



As we can observe we have improved the model by 0.004 on the f1 score. 

### Farther tuning ... 

Now we are going to tune the other parameters.   
(Notice that we are not tuning the aggresiveness parameter)

In [57]:
#parameters
parameters = {
    "algorithm": ['pa'],
    "max_iterations": [100],
    'pa_type': [0],
    'error_sensitive':[True,False],
    'averaging': [True,False] 
}

#tune
clf, results = train_tune_classifier(parameters, X_train, y_train, verbose=2)
results

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100, total=  22.7s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100, total=  22.8s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100, total=  22.6s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=100, total=  22.5s
[CV] averaging=True, pa_type=0, error_sensitive=False, algorithm=pa, max_iterations=100 
[CV]  averaging=True, pa_type=0, error_sensitive=False, algorithm=pa, max_iterations=10

[Parallel(n_jobs=4)]: Done  16 out of  16 | elapsed:  7.4min finished


Best Parameters: {'averaging': True, 'pa_type': 0, 'error_sensitive': True, 'algorithm': 'pa', 'max_iterations': 100}
Best Score: 0.7600


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,params
0,22.021102,0.625853,0.759988,0.967376,"True, 0, True, pa, 100"
1,20.543878,0.995777,0.757528,0.962168,"True, 0, False, pa, 100"
2,20.268769,0.636548,0.719321,0.940795,"False, 0, True, pa, 100"
3,20.257003,0.642844,0.711953,0.937266,"False, 0, False, pa, 100"


As it can be observed, the  of best scores are characterized for having the averaging parameter set to True. The two of them have a mean test score greater than 0.75 while the other ones are aroung 0.71.

Let's look how it performs on the test data.

In [58]:
#predict
y_pred = clf.predict(X_test)
print metrics.flat_classification_report(y_test, y_pred, labels=sorted(labels), digits=3 )

             precision    recall  f1-score   support

      B-LOC      0.800     0.753     0.776      1084
     B-MISC      0.683     0.566     0.619       339
      B-ORG      0.815     0.841     0.827      1400
      B-PER      0.809     0.876     0.841       735
      I-LOC      0.730     0.640     0.682       325
     I-MISC      0.707     0.562     0.626       557
      I-ORG      0.855     0.793     0.823      1104
      I-PER      0.885     0.935     0.910       634

avg / total      0.804     0.780     0.790      6178



So there a slight improvement on the precision score. 

### Tuning number of rounds

Finally we are going to tune the number of rounds or max_iterations. We are going to test on a wide range of iterations to see how the scores change as we increase the number of iterations.

In [59]:
#parameters
parameters = {
    "algorithm": ['pa'],
    "max_iterations": [30,50,80,100,200,400],
    'pa_type': [0],
    'error_sensitive':[True],
    'averaging': [True] 
}

#tune
clf, results = train_tune_classifier(parameters, X_train, y_train, verbose=2)
results

Fitting 4 folds for each of 6 candidates, totalling 24 fits
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30, total=  10.0s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30, total=   9.6s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30, total=  10.6s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=30, total=   9.8s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=50 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=50, total=  1

[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed: 20.0min finished


Best Parameters: {'averaging': True, 'pa_type': 0, 'error_sensitive': True, 'algorithm': 'pa', 'max_iterations': 80}
Best Score: 0.7594


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,params
2,17.122290,0.629203,0.759442,0.964307,"True, 0, True, pa, 80"
3,20.343756,0.627374,0.759349,0.967521,"True, 0, True, pa, 100"
1,12.748758,1.033526,0.759193,0.956491,"True, 0, True, pa, 50"
0,9.379437,0.620304,0.758696,0.941845,"True, 0, True, pa, 30"
4,160.366154,0.948598,0.754008,0.973328,"True, 0, True, pa, 200"
5,78.943230,0.941614,0.749681,0.975045,"True, 0, True, pa, 400"


As we can observe the best number of iterations is 80. A bigger number and the 
Let's evaluate the model against the test data.

In [60]:
#predict
y_pred = clf.predict(X_test)
print metrics.flat_classification_report(y_test, y_pred, labels=sorted(labels), digits=3 )

             precision    recall  f1-score   support

      B-LOC      0.792     0.757     0.774      1084
     B-MISC      0.690     0.552     0.613       339
      B-ORG      0.809     0.836     0.822      1400
      B-PER      0.808     0.868     0.837       735
      I-LOC      0.720     0.649     0.683       325
     I-MISC      0.726     0.548     0.624       557
      I-ORG      0.853     0.804     0.828      1104
      I-PER      0.886     0.934     0.909       634

avg / total      0.803     0.779     0.789      6178



So actually the model overfitted a little bit and we got a slightly lower score. Probably we will need to relax the tuning a little bit by trial and error.

In [61]:
#parameters
parameters = {
    "algorithm": ['pa'],
    "max_iterations": [70,90,100],
    'pa_type': [0],
    'error_sensitive':[True],
    'averaging': [True] 
}

#tune
clf, results = train_tune_classifier(parameters, X_train, y_train, verbose=2)
results

Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70, total=  21.3s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70, total=  18.3s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70, total=  17.7s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=70, total=  19.0s
[CV] averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=90 
[CV]  averaging=True, pa_type=0, error_sensitive=True, algorithm=pa, max_iterations=90, total=  2

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  6.0min finished


Best Parameters: {'averaging': True, 'pa_type': 0, 'error_sensitive': True, 'algorithm': 'pa', 'max_iterations': 100}
Best Score: 0.7598


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,params
2,22.597318,0.655407,0.759790,0.967175,"True, 0, True, pa, 100"
0,18.392606,0.680045,0.759365,0.962603,"True, 0, True, pa, 70"
1,21.765723,1.450070,0.759000,0.966271,"True, 0, True, pa, 90"


In [62]:
#predict
y_pred = clf.predict(X_test)
print metrics.flat_classification_report(y_test, y_pred, labels=sorted(labels), digits=3 )

             precision    recall  f1-score   support

      B-LOC      0.802     0.757     0.779      1084
     B-MISC      0.680     0.563     0.616       339
      B-ORG      0.810     0.843     0.826      1400
      B-PER      0.815     0.873     0.843       735
      I-LOC      0.725     0.634     0.677       325
     I-MISC      0.713     0.558     0.626       557
      I-ORG      0.854     0.798     0.825      1104
      I-PER      0.889     0.931     0.909       634

avg / total      0.805     0.781     0.791      6178



Great, now we had a slightly better score. In the end we improved the model by 0.005 on the f1-score. Not much, but good enough for the features we used.

# Conclusion
We've learned how to train and tune a CRF using the sklearn-crfsuite library. With it we extracted the labels for named entities.  

Next steps would be to create better features, select the best ones, tune again the model and evaluate the score.